# Project 4: Software Development Tools for Analysis of Vehicle Sales

For this project I'll be analyzing the vehicles_us dataset. I'll be looking to answer 3 questions from this dataset:
1: Do 4WD vehicles sell faster than non-4WD vehicles?
2: What color cars sell for the most on average?
3: Do sedans/coupes with high mileage sell for more than SUV's/trucks with high mileage?
Assumptions made for this analysis is that the price is in USD, "high mileage" is any odometer reading over 100,000 miles, and that every car was sold on the last day it was listed (highest possible days_listed value). 

In [15]:
import streamlit as st
import pandas as pd
import plotly.express as px
import numpy as np

In [16]:
df = pd.read_csv('C:/Users/turne/Project4/vehicles_us.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51525 entries, 0 to 51524
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   price         51525 non-null  int64  
 1   model_year    47906 non-null  float64
 2   model         51525 non-null  object 
 3   condition     51525 non-null  object 
 4   cylinders     46265 non-null  float64
 5   fuel          51525 non-null  object 
 6   odometer      43633 non-null  float64
 7   transmission  51525 non-null  object 
 8   type          51525 non-null  object 
 9   paint_color   42258 non-null  object 
 10  is_4wd        25572 non-null  float64
 11  date_posted   51525 non-null  object 
 12  days_listed   51525 non-null  int64  
dtypes: float64(4), int64(2), object(7)
memory usage: 5.1+ MB


In [17]:
#Altering data types for use later on in the project
#df['model_year'] = pd.to_numeric(df['model_year'])
#df['model_year'] = df['model_year'].astype('Int64')
df['cylinders'] = pd.to_numeric(df['cylinders'])
df['cylinders'] = df['cylinders'].astype('Int64')
df['date_posted'] = pd.to_datetime(df['date_posted'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51525 entries, 0 to 51524
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   price         51525 non-null  int64         
 1   model_year    47906 non-null  float64       
 2   model         51525 non-null  object        
 3   condition     51525 non-null  object        
 4   cylinders     46265 non-null  Int64         
 5   fuel          51525 non-null  object        
 6   odometer      43633 non-null  float64       
 7   transmission  51525 non-null  object        
 8   type          51525 non-null  object        
 9   paint_color   42258 non-null  object        
 10  is_4wd        25572 non-null  float64       
 11  date_posted   51525 non-null  datetime64[ns]
 12  days_listed   51525 non-null  int64         
dtypes: Int64(1), datetime64[ns](1), float64(3), int64(2), object(6)
memory usage: 5.2+ MB


In [18]:
#Checking for missing and duplicate data in each column
print(f"There are {df.duplicated().sum()} duplicated entries in this dataset.")
print()
print("Missing entries:")
print(df.isna().sum())

There are 0 duplicated entries in this dataset.

Missing entries:
price               0
model_year       3619
model               0
condition           0
cylinders        5260
fuel                0
odometer         7892
transmission        0
type                0
paint_color      9267
is_4wd          25953
date_posted         0
days_listed         0
dtype: int64


In [19]:
#Cleaning up and filling in missing data
df['model_year'] = df['model_year'].fillna('Unknown')
df['cylinders'] = df['cylinders'].fillna(0)
df['odometer'] = df['odometer'].fillna(0)
df['paint_color'] = df['paint_color'].fillna('Unknown')
df['is_4wd'] = df['is_4wd'].replace(np.nan, 0).astype(bool)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51525 entries, 0 to 51524
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   price         51525 non-null  int64         
 1   model_year    51525 non-null  object        
 2   model         51525 non-null  object        
 3   condition     51525 non-null  object        
 4   cylinders     51525 non-null  Int64         
 5   fuel          51525 non-null  object        
 6   odometer      51525 non-null  float64       
 7   transmission  51525 non-null  object        
 8   type          51525 non-null  object        
 9   paint_color   51525 non-null  object        
 10  is_4wd        51525 non-null  bool          
 11  date_posted   51525 non-null  datetime64[ns]
 12  days_listed   51525 non-null  int64         
dtypes: Int64(1), bool(1), datetime64[ns](1), float64(1), int64(2), object(7)
memory usage: 4.8+ MB


In [20]:
#Added manufacturer column for later analysis
df['manufacturer'] = df['model'].apply(lambda x: x.split()[0].title())
df.head(10)

,price,model_year,model,condition,cylinders,fuel,odometer,transmission,type,paint_color,is_4wd,date_posted,days_listed,manufacturer
0,9400,2011.0,bmw x5,good,6,gas,145000.0,automatic,SUV,Unknown,True,2018-06-23,19,Bmw
1,25500,Unknown,ford f-150,good,6,gas,88705.0,automatic,pickup,white,True,2018-10-19,50,Ford
2,5500,2013.0,hyundai sonata,like new,4,gas,110000.0,automatic,sedan,red,False,2019-02-07,79,Hyundai
3,1500,2003.0,ford f-150,fair,8,gas,0.0,automatic,pickup,Unknown,False,2019-03-22,9,Ford
4,14900,2017.0,chrysler 200,excellent,4,gas,80903.0,automatic,sedan,black,False,2019-04-02,28,Chrysler
5,14990,2014.0,chrysler 300,excellent,6,gas,57954.0,automatic,sedan,black,True,2018-06-20,15,Chrysler
6,12990,2015.0,toyota camry,excellent,4,gas,79212.0,automatic,sedan,white,False,2018-12-27,73,Toyota
7,15990,2013.0,honda pilot,excellent,6,gas,109473.0,automatic,SUV,black,True,2019-01-07,68,Honda
8,11500,2012.0,kia sorento,excellent,4,gas,104174.0,automatic,SUV,Unknown,True,2018-07-16,19,Kia
9,9200,2008.0,honda pilot,excellent,0,gas,147191.0,automatic,SUV,blue,True,2019-02-15,17,Honda


### Comparison of 4WD vs Non-4WD Vehicle Sale Time

Null hypothesis: 4WD and Non-4WD vehicles have the same average vehicle sale time

Alternative hypothesis: 4WD and Non-4WD vehicles do not have the same average vehicle sale time

In [21]:
avg_sale_pivot = df.pivot_table(index = 'is_4wd', values = ['days_listed'], aggfunc = ['mean', 'std', 'var'])
avg_sale_pivot = avg_sale_pivot.round(2)
print("Pivot Table: Sale Time for 4WD vs Non-4WD Vehicles")
avg_sale_pivot

Pivot Table: Sale Time for 4WD vs Non-4WD Vehicles


,mean,std,var
,days_listed,days_listed,days_listed
is_4wd,,,
False,39.50,28.32,802.29
True,39.61,28.08,788.59


In [14]:
st.header('4WD vs Non-4WD Vehicle Sale Time')
fig_4WD = px.histogram(df, x='days_listed', color='is_4wd', title='Sale Time for 4WD vs Non-4WD Vehicles',
                   labels={'days_listed': 'Days Listed', 'is_4wd': '4WD Vehicle'},
                   nbins=30)
st.write(fig_4WD)

2025-03-18 19:29:09.703 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-18 19:29:14.114 
  command:

    streamlit run c:\Users\turne\Project4\env\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-03-18 19:29:14.116 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-18 19:29:14.527 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-18 19:29:14.529 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-18 19:29:14.531 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-18 19:29:14.533 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


### Comparing Average Sale Prices of Vehicles Based on Paint Color

Null hypothesis: Car paint color does not affect average vehicle sale price

Alternative hypothesis: Car paint color does affect average vehicle sale price

In [22]:
paint_sale_pivot = df.pivot_table(index = 'paint_color', values = ['price'], aggfunc = ['mean', 'std', 'var'])
paint_sale_pivot = paint_sale_pivot.round(2)
paint_sale_pivot

,mean,std,var
,price,price,price
paint_color,,,
Unknown,12170.75,9599.84,9.215695e+07
black,13741.89,11239.54,1.263272e+08
blue,10269.71,10072.41,1.014534e+08
brown,10193.03,8621.82,7.433571e+07
custom,10030.10,7946.43,6.314573e+07
green,8426.98,7347.24,5.398201e+07
grey,11459.56,10107.45,1.021605e+08
orange,14654.35,9466.79,8.962013e+07


In [23]:
avg_price_by_paint = df.groupby('paint_color')['price'].mean().reset_index()

fig_paint = px.bar(avg_price_by_paint,
                   x = 'paint_color',
                   y = 'price',
                   title = 'Average Sale Price by Paint Color (USD)',
                   labels = {'paint_color': 'Paint Color', 'price': 'Average Price ($)'},
                   color = 'paint_color',
                    color_discrete_map={
                       'white': 'lightblue', 'red': 'red', 'black': 'black', 'blue': 'blue', 'grey': 'grey', 'silver': 'silver', 
                       'custom': 'gold', 'orange': 'orange', 'yellow': 'yellow', 'brown': 'brown', 'green': 'green', 'purple': 'purple'})
fig_paint.show()

### Comparing Average Sale Prices of High Mileage (Odometer Reading > 100,000) Sedans/Coupes vs. Pickup Trucks/SUVs

Null hypothesis: High mileage sedans and coupes have the same average sale price as high mileage pickup trucks and SUVs

Alternative hypothesis: High mileage sedans and coupes do not have the same average sale price as high mileage pickup trucks and SUVs

In [ ]:
#Find all vehicles with odometer readings > 100,000 miles
high_mileage = df[df['odometer'] > 100000]
#Filter resulting df to vehicle types SUV, sedan, pickup, coupe
high_mileage_of_interest = high_mileage[high_mileage['type'].isin(['SUV', 'sedan', 'pickup', 'coupe'])]
#Group by type and compare average prices
avg_price_high_mileage = high_mileage_of_interest.groupby('type')['price'].mean().reset_index()
#avg_price_high_mileage

high_mileage_pivot = high_mileage.pivot_table(index = 'type', values = ['price'], aggfunc = ['mean', 'std', 'var'])
high_mileage_pivot = high_mileage_pivot.round(2)
high_mileage_pivot